In [1]:
import pickle
import os
import numpy as np
import sklearn.metrics as metrics
import matplotlib.pyplot as plt

from tensorflow.keras.callbacks import ModelCheckpoint,CSVLogger
from tensorflow.keras.layers import Activation, Conv1D, Dense, Embedding, Flatten, Input, MaxPooling1D,LSTM,Dropout,SpatialDropout1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import categorical_accuracy
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.optimizers import RMSprop


Using TensorFlow backend.


In [2]:
#Processed Data Frame [ Do Not Use this, Use the Training, Tesing and Validation Data Sets Instead]
data_path = "./Pickles/all_articles_processed.pickle"
with open(data_path, 'rb') as data:
    all_articles = pickle.load(data)
    
#TD-IDF Features    
#Training Features
training_features_path = "./Pickles/tdidf_training_features.pickle"
with open(training_features_path, 'rb') as data:
    tdidf_training_features = pickle.load(data)
    
#Training Labels
training_labels_path = "./Pickles/tdidf_training_labels.pickle"
with open(training_labels_path, 'rb') as data:
    tdidf_training_labels = pickle.load(data)
    
#Test Features
test_features_path = "./Pickles/tdidf_test_features.pickle"
with open(test_features_path, 'rb') as data:
    tdidf_test_features = pickle.load(data)
    
#Test Labels
test_labels_path = "./Pickles/tdidf_test_labels.pickle"
with open(test_labels_path, 'rb') as data:
    tdidf_test_labels = pickle.load(data)
    
#Validation Features
test_features_path = "./Pickles/tdidf_validation_features.pickle"
with open(test_features_path, 'rb') as data:
    tdidf_validation_features = pickle.load(data)
    
#Validation Labels
test_labels_path = "./Pickles/tdidf_validation_labels.pickle"
with open(test_labels_path, 'rb') as data:
    tdidf_validation_labels = pickle.load(data)
    
    
#Sequence Vector Features    
#Training Features
training_features_path = "./Pickles/sv_training_features.pickle"
with open(training_features_path, 'rb') as data:
    sv_training_features = pickle.load(data)
    
#Training Labels
training_labels_path = "./Pickles/sv_training_labels.pickle"
with open(training_labels_path, 'rb') as data:
    sv_training_labels = pickle.load(data)
    
#Test Features
test_features_path = "./Pickles/sv_test_features.pickle"
with open(test_features_path, 'rb') as data:
    sv_test_features = pickle.load(data)
    
#Test Labels
test_labels_path = "./Pickles/sv_test_labels.pickle"
with open(test_labels_path, 'rb') as data:
    sv_test_labels = pickle.load(data)
    
#Validation Features
test_features_path = "./Pickles/sv_validation_features.pickle"
with open(test_features_path, 'rb') as data:
    sv_validation_features = pickle.load(data)
    
#Validation Labels
test_labels_path = "./Pickles/sv_validation_labels.pickle"
with open(test_labels_path, 'rb') as data:
    sv_validation_labels = pickle.load(data)

In [3]:
all_articles

,source,title,article,category,length_characters,length_words,category_code
0,AsiaOne,7 factors to consider when looking for an HDB ...,whether youre buy hdb resale flat firsttimer l...,Lifestyle,9771,1769,3
1,The Straits Times,Jung Joon-young first to be charged in K-pop s...,seoul first arrest kpop scandal singer jung ...,Lifestyle,2162,324,3
2,The Straits Times,Music mogul Dr Dre gets flak over boast that h...,los angeles look talk lack class netizens im...,Lifestyle,1108,182,3
3,Channel News Asia,NaN,think colonel sanders quirky enough apparentl...,Lifestyle,1193,207,3
4,Channel News Asia,NaN,mexican government question louis vuittons use...,Lifestyle,1484,231,3
5,Channel News Asia,NaN,style superstar virgil abloh produce one big m...,Lifestyle,3572,588,3
6,The Straits Times,Jerry Yan brightens up for actress Chen Mei-fe...,taiwanese actor jerry yan look downcast appea...,Lifestyle,1275,212,3
7,The Straits Times,Andy Hui scandal: Hong Kongers worry about the...,hong kong taxi driver break law sell minute ...,Lifestyle,2917,469,3
8,The Straits Times,Angel Has Fallen tops North America box office...,new york ap angel fall easily top box office...,Lifestyle,1380,232,3
9,The New Paper,Jackson unleashes fury over Nick Fury eye patc...,los angeles samuel l jackson unleash fury er...,Lifestyle,1227,198,3


In [4]:
df=all_articles[['article','category']]
df.head()

,article,category
0,whether youre buy hdb resale flat firsttimer l...,Lifestyle
1,seoul first arrest kpop scandal singer jung ...,Lifestyle
2,los angeles look talk lack class netizens im...,Lifestyle
3,think colonel sanders quirky enough apparentl...,Lifestyle
4,mexican government question louis vuittons use...,Lifestyle


In [5]:
df['category'].value_counts()

Business      1200
Technology    1200
World         1200
Singapore     1200
Lifestyle     1200
Sports        1200
Name: category, dtype: int64

In [6]:
X = df.article
Y = df.category
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [7]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15, random_state=1)


In [8]:
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(6120,) (6120, 1)
(1080,) (1080, 1)


In [9]:
max_words = 50000
max_len = 250

# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 500
# This is fixed.
EMBEDDING_DIM = 100

tok = Tokenizer(num_words=MAX_NB_WORDS)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [10]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
word_index = tok.word_index
print('Found %s unique tokens.' % len(word_index))

Found 65584 unique tokens.


In [11]:
X = tokenizer.texts_to_sequences(X.values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (7200, 500)


In [12]:
print('Shape of data tensor:', Y.shape)

Shape of data tensor: (7200, 1)


In [13]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,EMBEDDING_DIM,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [14]:
def RNN():
    model = Sequential()
    model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=max_len))
    model.add(SpatialDropout1D(0.2))
    model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(1, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [15]:
def RNN():
    model = Sequential()
    model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=max_len))
    model.add(SpatialDropout1D(0.7))
    model.add(LSTM(64, dropout=0.7, recurrent_dropout=0.7))
    model.add(Dense(1, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [16]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

W0917 20:25:52.664946  7656 nn_ops.py:4283] Large dropout rate: 0.7 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W0917 20:25:52.706833  7656 nn_ops.py:4283] Large dropout rate: 0.7 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W0917 20:25:52.718803  7656 nn_ops.py:4283] Large dropout rate: 0.7 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W0917 20:25:52.730770  7656 nn_ops.py:4283] Large dropout rate: 0.7 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W0917 20:25:52.742715  7656 nn_ops.py:4283] Large dropout rate: 0.7 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W0917 20:25:52.756676  7656 nn_ops.py:4283] Large dropout rate: 0.7 (>0.5). In TensorFlow 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 250, 100)          5000000   
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 250, 100)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 64)                42240     
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 5,042,305
Trainable params: 5,042,305
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

W0917 20:25:52.977647  7656 nn_ops.py:4283] Large dropout rate: 0.7 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


Train on 4896 samples, validate on 1224 samples
Epoch 1/10


W0917 20:25:53.213577  7656 deprecation.py:323] From C:\Users\darry\AppData\Roaming\Python\Python37\site-packages\tensorflow_core\python\ops\math_grad.py:1394: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0917 20:25:53.773908  7656 nn_ops.py:4283] Large dropout rate: 0.7 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


4896/4896 [==============================] - 16s 3ms/sample - loss: -22.9404 - accuracy: 0.1673 - val_loss: -23.0249 - val_accuracy: 0.1683
Epoch 2/10
4896/4896 [==============================] - 15s 3ms/sample - loss: -22.9404 - accuracy: 0.1673 - val_loss: -23.0249 - val_accuracy: 0.1683
